##### Copyright 2020 The TensorFlow Authors.

In [ ]:
#@title Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
# https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Better ML Engineering with Machine Learning Metadata



<table class="tfo-notebook-buttons" align="left">
  <td>
    <a target="_blank" href="https://www.tensorflow.org/tfx/tutorials/mlmd_tutorial"><img src="https://www.tensorflow.org/images/tf_logo_32px.png" />View on TensorFlow.org</a>
  </td>
  <td>
    <a target="_blank" href="https://colab.research.google.com/github/tensorflow/tfx/blob/master/docs/tutorials/mlmd_tutorial.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png" />Run in Google Colab</a>
  </td>
  <td>
    <a target="_blank" href="https://github.com/tensorflow/docs/blob/master/tools/templates/notebook.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png" />View source on GitHub</a>
  </td>
  <td>
    <a href="https://storage.googleapis.com/tensorflow_docs/docs/tools/templates/notebook.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png" />Download notebook</a>
  </td>
</table>

Assume a scenario where you set up a production ML pipeline to classify pictures of iris flowers. The pipeline ingests your training data, trains and evaluates a model, and pushes it to production. 

However, when you later try using this model with a larger dataset that contains images of different kinds of flowers, you observe that your model does not behave as expected and starts classifying roses and lilies as types of irises.

At this point, you are interested in knowing:

* What is the most efficient way to debug the model when the only available artifact is the model in production?
* Which training dataset was used to train the model?
* Which training run led to this erroneous model?
* Where are the model evaluation results? 
* Where to begin debugging?

[Machine Learning Metadata (MLMD)](https://github.com/google/ml-metadata) is a library that leverages the metadata associated with ML models to help you answer these questions and more. MLMD enables you to reliably track the artifacts and lineage associated with the various components of your ML pipeline.   

In this tutorial, you set up a TFX Pipeline to create a models that classifies Iris flowers into three species - Iris setosa, Iris virginica, and Iris versicolor based on the length and width measurements of their petals and sepals. You then use MLMD to track the lineage of pipeline components.

## TFX Pipelines in Colab

Colab is a lightweight development environment which differs significantly from a production environment. In production, you may have various pipeline components like data ingestion, transformation, model training, run histories, etc. across multiple, distributed systems. For this tutorial, you should be aware that siginificant differences exist in Orchestration and Metadata storage - it is all handled locally within Colab. Learn more about TFX in Colab by reading the [TFX Keras Components tutorial](https://www.tensorflow.org/tfx/tutorials/tfx/components_keras#background)



## Setup

Import all required libraries.

### Install and import TFX

In [ ]:
!pip install --quiet tfx

You must restart the Colab runtime after installing TFX. Select **Runtime > Restart runtime** from the Colab menu.

### Import other libraries

In [ ]:
import base64
import csv
import json
import os
import requests
import tempfile
import urllib
import pprint
import numpy as np

In [ ]:
import tensorflow as tf
import tfx

Import [TFX component](https://tensorflow.google.cn/tfx/tutorials/tfx/components_keras) classes.

In [ ]:
from tfx.components.evaluator.component import Evaluator
from tfx.components.example_gen.csv_example_gen.component import CsvExampleGen
from tfx.components.pusher.component import Pusher
from tfx.components.schema_gen.component import SchemaGen
from tfx.components.statistics_gen.component import StatisticsGen
from tfx.components.trainer.component import Trainer
from tfx.components.transform.component import Transform
from tfx.components.base import executor_spec
from tfx.components.trainer.executor import GenericExecutor
from tfx.orchestration.experimental.interactive.interactive_context import InteractiveContext
from tfx.proto import evaluator_pb2
from tfx.proto import pusher_pb2
from tfx.proto import trainer_pb2
from tfx.utils.dsl_utils import external_input

from tensorflow_metadata.proto.v0 import anomalies_pb2
from tensorflow_metadata.proto.v0 import schema_pb2
from tensorflow_metadata.proto.v0 import statistics_pb2

import tensorflow_transform as tft
from tensorflow_transform import coders as tft_coders
from tensorflow_transform.tf_metadata import dataset_schema
from tensorflow_transform.tf_metadata import schema_utils

import tensorflow_model_analysis as tfma
import tensorflow_data_validation as tfdv

from tfx.components import ResolverNode
from tfx.dsl.experimental import latest_blessed_model_resolver
from tfx.types import Channel
from tfx.types.standard_artifacts import Model
from tfx.types.standard_artifacts import ModelBlessing

Import the MLMD library.

In [ ]:
import ml_metadata
from ml_metadata.metadata_store import metadata_store
from ml_metadata.proto import metadata_store_pb2

## Download the dataset

Download the [Iris dataset](https://archive.ics.uci.edu/ml/datasets/iris) dataset to use in this tutorial. The dataset consists of flower data for 150 Iris flowers belonging to one of three species - Iris setosa, Iris virginica, and Iris versicolor.

In [ ]:
DATA_PATH = 'https://raw.githubusercontent.com/tensorflow/tfx/master/tfx/examples/iris/data/iris.csv'
_data_root = tempfile.mkdtemp(prefix='tfx-data')
_data_filepath = os.path.join(_data_root, "iris.csv")
urllib.request.urlretrieve(DATA_PATH, _data_filepath)

## Create an InteractiveContext

To run TFX components interactively in this notebook, create an `InteractiveContext`. The `InteractiveContext` uses a temporary directory with an ephemeral MLMD database instance. Note that calls to `InteractiveContext` are no-ops outside the Colab environment.

In [ ]:
int_context = InteractiveContext()

## Construct the TFX Pipeline

A TFX pipeline consists of several components that perform different aspects of the ML workflow. In this notebook, you create and run the `ExampleGen`, `StatisticsGen`, `SchemaGen`, and `TrainerGen` components. Refer to the [components tutorial](https://www.tensorflow.org/tfx/tutorials/tfx/components_keras) for more information on TFX pipeline components.

### Instantiate and run the ExampleGen Component

In [ ]:
input_data = external_input(_data_root)
example_gen = CsvExampleGen(input=input_data)

# Run the ExampleGen component using the InteractiveContext
int_context.run(example_gen)

### Instantiate and run the StatisticsGen Component

In [ ]:
statistics_gen = StatisticsGen(examples=example_gen.outputs['examples'])

# Run the StatisticsGen component using the InteractiveContext
int_context.run(statistics_gen)

### Instantiate and run the SchemaGen Component

In [ ]:
infer_schema = SchemaGen(statistics=statistics_gen.outputs['statistics'],
                         infer_feature_shape = True)

# Run the SchemaGen component using the InteractiveContext
int_context.run(infer_schema)

### Instantiate and run the Trainer Component








In [ ]:
# Define the module file for the Trainer component
trainer_module_file = 'iris_trainer.py'

In [ ]:
# Define the training algorithm for the Trainer module file 
%%writefile {trainer_module_file}
import os
from typing import List, Text

import tensorflow as tf
from tensorflow import keras

from tfx.components.trainer.executor import TrainerFnArgs
from tfx.components.trainer.fn_args_utils import FnArgs

# Iris dataset has 150 records, and is split into training and evaluation 
# datasets in a 2:1 split

_TRAIN_DATA_SIZE = 100
_EVAL_DATA_SIZE = 50
_TRAIN_BATCH_SIZE = 100
_EVAL_BATCH_SIZE = 50

_FEATURES = {
    'sepal_length': tf.io.FixedLenFeature([], dtype=tf.float32, default_value=0),
    'sepal_width': tf.io.FixedLenFeature([], dtype=tf.float32, default_value=0),
    'petal_length': tf.io.FixedLenFeature([], dtype=tf.float32, default_value=0),
    'petal_width': tf.io.FixedLenFeature([], dtype=tf.float32, default_value=0),
    'variety': tf.io.FixedLenFeature([], dtype=tf.int64, default_value=0)
}

_LABEL_KEY = 'variety'

_FEATURE_KEYS = ['sepal_length', 'sepal_width', 'petal_length', 'petal_width']

def _gzip_reader_fn(filenames):
  return tf.data.TFRecordDataset(filenames, compression_type='GZIP')

def _input_fn(file_pattern: List[Text],
              batch_size: int = 200):
  dataset = tf.data.experimental.make_batched_features_dataset(
            file_pattern=file_pattern,
            batch_size=batch_size,
            features=_FEATURES,
            reader=_gzip_reader_fn,
            label_key=_LABEL_KEY)
  
  return dataset
  
def _build_keras_model():
  inputs = [keras.layers.Input(shape = (1,), name = f) for f in _FEATURE_KEYS]
  d = keras.layers.concatenate(inputs)
  d = keras.layers.Dense(8, activation = 'relu')(d)
  d = keras.layers.Dense(8, activation = 'relu')(d)
  outputs = keras.layers.Dense(3, activation = 'softmax')(d)
  model = keras.Model(inputs=inputs, outputs=outputs)
  model.compile(optimizer = 'adam',
                loss = 'sparse_categorical_crossentropy',
                metrics= [keras.metrics.SparseCategoricalAccuracy()])
  return model

def run_fn(fn_args: TrainerFnArgs):
  train_dataset = _input_fn(fn_args.train_files, batch_size=_TRAIN_BATCH_SIZE)
  eval_dataset = _input_fn(fn_args.eval_files, batch_size=_EVAL_BATCH_SIZE)
  
  model = _build_keras_model()

  print(model.summary())

  steps_per_epoch = _TRAIN_DATA_SIZE / _TRAIN_BATCH_SIZE

  model.fit(train_dataset, 
            epochs=int(fn_args.train_steps / steps_per_epoch),
            steps_per_epoch=steps_per_epoch,
            validation_data=eval_dataset,
            validation_steps=fn_args.eval_steps)
  model.save(fn_args.serving_model_dir, save_format='tf')

Run the `Trainer` component.

In [ ]:
trainer = Trainer(
    module_file=os.path.abspath(trainer_module_file),
    custom_executor_spec=executor_spec.ExecutorClassSpec(GenericExecutor),
    examples=example_gen.outputs['examples'],
    schema=infer_schema.outputs['schema'],
    train_args=trainer_pb2.TrainArgs(num_steps=100),
    eval_args=trainer_pb2.EvalArgs(num_steps=50))

int_context.run(trainer)

### Evaluate and push the model

Use the `Evaluator` component to evaluate and 'bless' the model before using the `Pusher` component to push the model to a serving directory.

In [ ]:
_serving_model_dir = os.path.join(tempfile.mkdtemp(), 'serving_model/iris_classification')

In [ ]:
eval_config = tfma.EvalConfig(model_specs=[tfma.ModelSpec(label_key ='variety')],
                              metrics_specs =[tfma.MetricsSpec(metrics = 
                                                               [tfma.MetricConfig(class_name='ExampleCount'),
                                                               tfma.MetricConfig(class_name='BinaryAccuracy',
                                                                  threshold=tfma.MetricThreshold(
                                                                      value_threshold=tfma.GenericValueThreshold(
                                                                          lower_bound={'value': 0.5}),
                                                                      change_threshold=tfma.GenericChangeThreshold(
                                                                          direction=tfma.MetricDirection.HIGHER_IS_BETTER,
                                                                          absolute={'value': -1e-10})))])],
                              slicing_specs = [tfma.SlicingSpec(),
                                               tfma.SlicingSpec(feature_keys=['sepal_length'])])

In [ ]:
model_resolver = ResolverNode(
      instance_name='latest_blessed_model_resolver',
      resolver_class=latest_blessed_model_resolver.LatestBlessedModelResolver,
      model=Channel(type=Model),
      model_blessing=Channel(type=ModelBlessing))
int_context.run(model_resolver)

evaluator = Evaluator(
    examples=example_gen.outputs['examples'],
    model=trainer.outputs['model'],
    baseline_model=model_resolver.outputs['model'],
    eval_config=eval_config)
int_context.run(evaluator)

In [ ]:
pusher = Pusher(
    model=trainer.outputs['model'],
    model_blessing=evaluator.outputs['blessing'],
    push_destination=pusher_pb2.PushDestination(
        filesystem=pusher_pb2.PushDestination.Filesystem(
            base_directory=_serving_model_dir)))
int_context.run(pusher)

Running the TFX pipeline populates the MLMD Database. In the next section, you use the MLMD API to query this database for metadata information. 

## Query the MLMD Database

The MLMD database stores three types of metadata: 

*    Metadata about the pipeline and lineage information associated with the pipeline components
*    Metadata about artifacts that were generated during the pipeline run
*    Metadata about the executions of the pipeline

Set up the metadata store with the `InteractiveContext` defined previously to query the MLMD database.

In [ ]:
md_store = metadata_store.MetadataStore(int_context.metadata_connection_config)

In a production environment, when you encounter erroneous results in a model pushed through your TFX pipeline, you can query the MLMD database to trace the lineage of your pipeline components and use it to find the artifacts that correspond to the pushed model. 

First, query the metadata store for pushed models:

In [ ]:
def get_pushed_models(store): 
  return store.get_artifacts_by_type("PushedModel")

# In this example, there is only one pushed model 
pushed_model = get_pushed_models(md_store)[0]
print(pushed_model)

Get the training data associated with this model. 

In [ ]:
def get_training_data(store, model):
  all_models = get_pushed_models(store)
  for models in all_models:
    if models.id == model.id:
      train_data = store.get_artifacts_by_type("Examples")
      return train_data
  return None

td = get_training_data(md_store, pushed_model)
if td is not None:
  print(td)

Now that you have the training data that the model trained with, query the database again to find the training step (execution):

In [ ]:
def get_training_step(store, model):
  all_models = get_pushed_models(store)
  for models in all_models:
    if models.id == model.id:
      train_step = store.get_executions_by_type("tfx.components.trainer.component.Trainer")
      return train_step
  return None

ts = get_training_step(md_store, pushed_model)
if ts is not None:
  print(ts)

You can also trace the lineage of the pushed model by querying all of its upstream artifacts and executions.

In [ ]:
pp = pprint.PrettyPrinter()

def get_upstream_artifacts_and_executions(artifact):
  artifact_types = execution_types = {} 
  for atype in md_store.get_artifact_types():
    artifact_types[atype.id] = atype.name
  for etype in md_store.get_execution_types():
    execution_types[etype.id] = etype.name

  q = artifact
  ra = {}
  re = {}
  ra[artifact_types[q.type_id]] = {q.id}
  eval_re = True
  while True:
    ra_len = len(ra)
    re_len = len(re)     
    if eval_re:
      eval_re = False
      executions = md_store.get_executions_by_id(
          [e.execution_id for e in md_store.get_events_by_artifact_ids(
              [aid for aid in sum([list(v) for v in ra.values()], [])]) 
          if e.type == metadata_store_pb2.Event.OUTPUT])
      for ex in executions:
        re.setdefault(execution_types[ex.type_id], set()).add(ex.id)       
    else:
      eval_re = True
      artifacts = md_store.get_artifacts_by_id(
          [e.artifact_id for e in md_store.get_events_by_execution_ids(
              [exid for exid in sum([list(v) for v in re.values()], [])]) 
          if e.type == metadata_store_pb2.Event.INPUT])
      for a in artifacts:
        ra.setdefault(artifact_types[a.type_id], set()).add(a.id)
    if (ra_len == len(ra) and re_len == len(re)):
      break
  return ra, re

ancestor_aid, ancestor_eid = get_upstream_artifacts_and_executions(pushed_model)
print("Upstream Artifacts `RA`:")
pp.pprint(ancestor_aid)
print("\nUpstream Executions `RE`:")
pp.pprint(ancestor_eid)

## Summary

In this tutorial, you learned how to use MLMD to analyze the lineage of the components of your TFX pipeline. To learn more about how to use MLMD, check out these additional resources:

* [MLMD API documentation](https://www.tensorflow.org/tfx/ml_metadata/api_docs/python/mlmd)
* [MLMD guide](https://www.tensorflow.org/tfx/guide/mlmd)